In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install xmltodict

In [ ]:
import os
import zipfile
import random
import xmltodict
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [ ]:
from google.colab import drive
import zipfile
import os

# Step 2: Define the path to the zip file and the extraction destination
zip_path = '/content/drive/MyDrive/Febrio (1)/base-model-box.v1i.coco.zip'
extract_to = '/content/dataset/'

# Step 3: Extract the zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extraction completed successfully to {extract_to}")
except FileNotFoundError:
    print("The specified zip file was not found. Please check the file path.")

# Step 4: List the contents of the extracted directory to confirm
print("Contents of the extracted directory:")
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")


Streaming output truncated to the last 5000 lines.
    cardboard1853_jpg.rf.3524594a6ac77e608bca4e2a538c59a3.jpg
    85_png_jpg.rf.6b9edbfd646037778941d91c516ed300.jpg
    im0-250-_jpg.rf.4c2ec3fb4f30c160788fe4c7407ff290.jpg
    im0-16-_jpg.rf.aacf9c950c98e0026d2bbae147f5ef2e.jpg
    cardboard1601_jpg.rf.c46d7b66e07fd20285ea9b1ef6dbec25.jpg
    imo-57-_jpg.rf.7198539a958098868537d5a3c43fbbad.jpg
    iiiiii-35-_jpg.rf.247e0d7ee2d82cb5c513d260d9a24fd9.jpg
    im0-122-_jpg.rf.66d492bf54cb07977c49d783c9d83337.jpg
    imo-37-_jpg.rf.6a746eb6fc5433ccf382fd7b89689170.jpg
    gggg-112-_jpg.rf.8f70246838b20d2d71a8b2f7ba62aaa9.jpg
    15-04-2022__10-49-01AM_jpg.rf.780ccf59675c954d043e8b4d21cbe676.jpg
    cardboard867_jpg.rf.4e28b9e69da854e9fa1c89756750b67d.jpg
    17_png_jpg.rf.3c0468d94daee6611c7f85ed613c520d.jpg
    iiiiii-185-_jpg.rf.5009325e8f73d3c01ffc54e493653896.jpg
    cardboard-1673_jpg.rf.6301c43d0a7282f76f007e23fbe27e70.jpg
    gggg-44-_jpg.rf.60bb09d278bc0b6d1bff53ec0a9e2f9e.jpg
    

In [ ]:
import json
import os
import shutil

# Path ke dataset
dataset_path = '/content/dataset/'
output_path = '/content/yolo_dataset/'

# Buat struktur direktori output YOLO
os.makedirs(os.path.join(output_path, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'images/test'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/test'), exist_ok=True)

# Fungsi untuk konversi bounding box COCO ke YOLO
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = box[0] + box[2] / 2.0
    y = box[1] + box[3] / 2.0
    w = box[2]
    h = box[3]
    return (x * dw, y * dh, w * dw, h * dh)

# Proses file JSON COCO (train dan test)
for subset in ['train', 'test']:
    coco_file_path = os.path.join(dataset_path, subset, '_annotations.coco.json')
    images_dir = os.path.join(dataset_path, subset)
    output_images_dir = os.path.join(output_path, f'images/{subset}')
    output_labels_dir = os.path.join(output_path, f'labels/{subset}')

    # Buka file COCO
    with open(coco_file_path, 'r') as f:
        coco_data = json.load(f)

    # Parse data COCO
    images = {img['id']: img for img in coco_data['images']}
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    annotations = coco_data['annotations']

    # Proses anotasi
    for ann in annotations:
        image_id = ann['image_id']
        category_id = ann['category_id']
        bbox = ann['bbox']

        # Ambil informasi gambar
        image_info = images[image_id]
        image_name = image_info['file_name']
        width, height = image_info['width'], image_info['height']

        # Konversi bounding box
        yolo_bbox = convert_bbox((width, height), bbox)

        # Path untuk file label YOLO
        label_file_path = os.path.join(output_labels_dir, f"{os.path.splitext(image_name)[0]}.txt")
        with open(label_file_path, 'a') as label_file:
            label_file.write(f"{category_id - 1} {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n")

        # Salin gambar ke folder YOLO
        src_image_path = os.path.join(images_dir, image_name)
        dest_image_path = os.path.join(output_images_dir, image_name)
        if not os.path.exists(dest_image_path):
            shutil.copy(src_image_path, dest_image_path)

print(f"Konversi selesai! Dataset YOLO tersedia di {output_path}")


Konversi selesai! Dataset YOLO tersedia di /content/yolo_dataset/


In [ ]:
import json
import os

# Path ke file COCO
train_coco_path = '/content/dataset/train/_annotations.coco.json'
test_coco_path = '/content/dataset/test/_annotations.coco.json'
output_yaml_path = '/content/yolo_dataset/data.yaml'

# Fungsi untuk membaca kategori dari file COCO
def get_categories(coco_file):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)
    categories = coco_data['categories']
    class_names = [cat['name'] for cat in categories]
    return class_names

# Ambil nama kelas dari file train (diasumsikan train dan test memiliki kategori yang sama)
class_names = get_categories(train_coco_path)

# Buat isi file data.yaml
data_yaml_content = {
    'train': '/content/yolo_dataset/images/train',
    'val': '/content/yolo_dataset/images/test',
    'nc': len(class_names),  # Jumlah kelas
    'names': class_names     # Nama kelas
}

# Pastikan direktori yolo_dataset ada
os.makedirs(os.path.dirname(output_yaml_path), exist_ok=True)

# Tulis file data.yaml di folder yolo_dataset
with open(output_yaml_path, 'w') as yaml_file:
    yaml_file.write(f"train: {data_yaml_content['train']}\n")
    yaml_file.write(f"val: {data_yaml_content['val']}\n")
    yaml_file.write(f"nc: {data_yaml_content['nc']}\n")
    yaml_file.write("names: [\n")
    for name in data_yaml_content['names']:
        yaml_file.write(f"  '{name}',\n")
    yaml_file.write("]\n")

print(f"File data.yaml berhasil dibuat di {output_yaml_path}")

File data.yaml berhasil dibuat di /content/yolo_dataset/data.yaml


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 51.4 MB/s eta 0:00:00


In [ ]:
 from ultralytics import YOLO

# Path ke file data.yaml
data_yaml_path = '/content/yolo_dataset/data.yaml'

# Buat model YOLOv8x
model = YOLO('yolov8x.pt')  # Pre-trained YOLOv8x model

# Latih model dengan dataset Anda
model.train(
    data=data_yaml_path,       # Path ke file data.yaml
    epochs=20,                 # Jumlah epoch (ubah sesuai kebutuhan)
    imgsz=640,                 # Ukuran gambar (640x640 default)
    batch=16,                  # Ukuran batch
    name='yolo8x-custom',      # Nama folder output
    save=True                  # Simpan model hasil pelatihan
)

# Evaluasi hasil pelatihan
metrics = model.val()
print("Hasil evaluasi:", metrics)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 131M/131M [00:03<00:00, 38.6MB/s]


Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/yolo_dataset/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo8x-custom, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

100%|██████████| 755k/755k [00:00<00:00, 99.8MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

...


20 epochs completed in 4.152 hours.
Optimizer stripped from runs/detect/yolo8x-custom/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/yolo8x-custom/weights/best.pt, 136.7MB

Validating runs/detect/yolo8x-custom/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68,126,457 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.22it/s]


                   all         14         23      0.847          1      0.949      0.809
              box-LRZb         14         23      0.847          1      0.949      0.809
Speed: 0.1ms preprocess, 17.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/yolo8x-custom
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68,126,457 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /content/yolo_dataset/labels/test.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


                   all         14         23      0.847          1      0.949      0.815
              box-LRZb         14         23      0.847          1      0.949      0.815
Speed: 0.2ms preprocess, 40.6ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/yolo8x-custom2
Hasil evaluasi: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x790ec4c49ff0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024

In [ ]:
import os

results_dir = '/content/runs/detect'
if os.path.exists(results_dir):
    for root, dirs, files in os.walk(results_dir):
        print(f"Path: {root}")
        for file in files:
            print(f" - {file}")
else:
    print("Folder hasil pelatihan tidak ditemukan.")


Path: /content/runs/detect
Path: /content/runs/detect/yolo8x-custom2
 - val_batch0_pred.jpg
 - confusion_matrix.png
 - confusion_matrix_normalized.png
 - R_curve.png
 - val_batch0_labels.jpg
 - PR_curve.png
 - P_curve.png
 - F1_curve.png
Path: /content/runs/detect/yolo8x-custom
 - val_batch0_pred.jpg
 - results.csv
 - confusion_matrix.png
 - labels.jpg
 - labels_correlogram.jpg
 - train_batch5951.jpg
 - train_batch2.jpg
 - confusion_matrix_normalized.png
 - R_curve.png
 - train_batch5950.jpg
 - val_batch0_labels.jpg
 - args.yaml
 - train_batch5952.jpg
 - train_batch0.jpg
 - train_batch1.jpg
 - PR_curve.png
 - events.out.tfevents.1731932839.20141a3fa358.470.0
 - P_curve.png
 - results.png
 - F1_curve.png
Path: /content/runs/detect/yolo8x-custom/weights
 - best.pt
 - last.pt


In [ ]:
from google.colab import files

# Specify the path of the model
model_path = '/content/runs/detect/train/yolo8x-custom/best.pt'

# Download the model
files.download(model_path)


FileNotFoundError: Cannot find file: /content/runs/detect/train/yolo8x-custom/best.pt